In [13]:
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.pardir)
import glob
import numpy as np
from multiprocessing import Pool
import multiprocessing as multi
sys.path.append(os.pardir)
import pandas as pd
import h5py
import re

In [2]:
file = h5py.File( '/home/dl-box/デスクトップ/PythonFile/nishimoto/SampleFullsn/fullsnspmStimW2V.mat', 'r' )
list( file.keys() )

['#refs#',
 'modelName',
 'stimW2VTrn',
 'stimW2VVal',
 'u',
 'wordArray',
 'wordFreqArray']

In [3]:
f = open('/home/dl-box/デスクトップ/PythonFile/nishimoto/SampleFullsn/WordList.txt', 'r')
w = f.read()
f.close()
wList = re.sub( '\n', ' ',w ).split()

In [4]:
np.array(file['u']).shape

(109732, 1000)

In [6]:
load_path = sorted( glob.glob( '/home/dl-box/デスクトップ/PythonFile/hirose/Text/text_list/*.txt' ) )

In [11]:
f = open(load_path[0],"r")
w = f.read()

In [4]:
# 頻出単語はカット
wList = wList[100:]

In [5]:

'''text2index'''

from func import TextDeal
from func import wrapper_func_four

vocab = wList

td = TextDeal()
load_path = sorted( glob.glob( '/home/dl-box/デスクトップ/PythonFile/hirose/Text/text_list/*.txt' ) )
name = [ os.path.splitext( os.path.basename( r ) )[0] for r in load_path ]
save_path = '/home/dl-box/デスクトップ/PythonFile/hirose/LSTM/W2I/'

if __name__ == '__main__':
    #data=(関数, 引数)
    data = [ ( td.text2index_nishimoto, vocab, load_path[i], save_path, name[i] ) for i in range( len(load_path) ) ]
    with Pool( multi.cpu_count()-1 ) as p:
        p.map( wrapper_func_four, data )
    

Using TensorFlow backend.


In [6]:
load_path = sorted( glob.glob( '/home/dl-box/デスクトップ/PythonFile/hirose/LSTM/W2I/*.npy' ) )
a = [ np.load(path) for path in load_path ]

In [7]:
q = []
w = []
for b in a:
    d=0
    w.append(len(b))
    for c in b:
        if d<len(c):
            d = len(c)
    q.append(d)

In [8]:
print( '各文の長さの最大値 > 25 (7440中) ', len(np.where(np.array(q)>25)[0]) )
print( '　各文の数の最大値 > 20 (7440中) ',len(np.where(np.array(w)>20)[0]) )

各文の長さの最大値 > 25 (7440中)  23
　各文の数の最大値 > 20 (7440中)  208


In [9]:

''' Padding '''

from func import TextDeal
from func import wrapper_func_three

td = TextDeal()
length = len( glob.glob( '/home/dl-box/デスクトップ/PythonFile/hirose/LSTM/W2I/*.npy' ) )
load_path = sorted( glob.glob( '/home/dl-box/デスクトップ/PythonFile/hirose/LSTM/W2I/*.npy' ) )
name = [ os.path.basename( r ) for r in load_path ]
save_path = '/home/dl-box/デスクトップ/PythonFile/hirose/LSTM/Padding/'

if __name__ == '__main__':
    #data=(関数, 引数)
    data = [ ( td.padding_plus1, load_path[i], save_path, name[i] ) for i in range( length ) ]
    with Pool( multi.cpu_count()-1 ) as p:
        p.map( wrapper_func_three, data )


In [11]:
#ちゃんとPaddingできたか確認
path = [ ('./Padding/'+os.path.basename( r )) for r in sorted( glob.glob( './Padding/*.npy' ) ) ]
for i in range(7740):
    if np.load( path[i] ).shape==(20,25):
        continue
    else:
        print('Error!!')
        break

In [12]:

''' Embedding Matrix '''

matW = np.array( file['u'] )[100:] #頻出100単語をカット
zero = np.array( [ np.zeros(1000,dtype=int) ] )
matW_New = np.append(zero, matW, axis=0)
np.save( './data/weights.npy', matW_New )

In [13]:
print('元々の形=' +str(matW.shape)+' 0加えたあと='+str(matW_New.shape) )

元々の形=(109632, 1000) 0加えたあと=(109633, 1000)


In [ ]:

''' Make Training Data & Test Data '''


In [21]:
#load する data
path = [ ('/home/dl-box/デスクトップ/PythonFile/hirose/Text/text_index/'+os.path.basename( r ))
          for r in sorted( glob.glob( '/home/dl-box/デスクトップ/PythonFile/hirose/Text/text_index/*.npy' ) ) ]

In [22]:
tr = []
te = []
for i in range( len(path) ):
    if i < 7200:
        tr.append( np.load(path[i]) )
        if i ==7199:
            print('training data finished')
    else:
        te.append( np.load(path[i]) )
        if i == 7739:
            print('test data finished')
        
    if i%1000 == 0:
            print( str(i) + '番目まで終了' )

0番目まで終了
1000番目まで終了
2000番目まで終了
3000番目まで終了
4000番目まで終了
5000番目まで終了
6000番目まで終了
7000番目まで終了
training data finished
test data finished


In [23]:
print('training size = ' + str(len(tr)) + ' test size = ' + str(len(te)) )

training size = 7200 test size = 540


In [24]:
np.save( './data/training_data.npy', tr )
np.save( './data/test_data.npy', te )

In [ ]:

''' timescale matrix '''


In [27]:
tr = np.load( './data/training_data.npy' )
te = np.load( './data/test_data.npy' )
tr.shape

(7200,)

In [11]:
'''zero = np.zeros_like( tr[0] )
#脳情報データラベルは3600個(training)
tr_data = []
for i in range(3600):
    tmp = []
    if i == 0:
        tmp.append( tr[0] )
        tmp.append( zero )
        tmp.append( zero )
    elif i == 1:
        tmp.append( tr[1] )
        tmp.append( tr[0] )
        tmp.append( zero )
    else:
        tmp.append( tr[2*i] )
        tmp.append( tr[2*i-1] )
        tmp.append( tr[2*i-2] )
    tr_data.append( tmp )
    if i % 1000 == 0:
        print(i)
    
te_data = []
for i in range(270):
    tmp = []
    if i == 0:
        tmp.append( tr[len(tr)-1] )
        tmp.append( tr[-1] )
        tmp.append( tr[-2] )
    elif i == 1:
        tmp.append( te[1] )
        tmp.append( te[0] )
        tmp.append( tr[len(tr)-1] )
    else:
        tmp.append( tr[2*i] )
        tmp.append( tr[2*i-1] )
        tmp.append( tr[2*i-2] )
    te_data.append( tmp )
    '''

0
1000
2000
3000


In [28]:
#脳情報データラベルは3600個(training)
tr_data = []
for i in range( 3, 3600+1 ): #最初の5秒間は捨てる(3599個)
    tmp = []
    
    tmp.append( tr[(2*i-1)-3] )
    tmp.append( tr[(2*i-1)-4] )
    tmp.append( tr[(2*i-1)-5] )
    
    tr_data.append( tmp )
    if i % 1000 == 0:
        print(i)
    
te_data = []
for i in range( 3, 270+1 ):
    tmp = []
    
    tmp.append( te[(2*i-1)-3] )
    tmp.append( te[(2*i-1)-4] )
    tmp.append( te[(2*i-1)-5] )
    
    te_data.append( tmp )
    

1000
2000
3000


In [12]:
'''
np.save( './lstm/tr_data(234).npy', tr_data )
np.save( './lstm/te_data(234).npy', te_data )
np.load('./lstm/tr_data(234).npy').shape
'''

In [29]:
np.save( './data/tr_data(345).npy', tr_data )
np.save( './data/te_data(345).npy', te_data )
np.load('./data/te_data(345).npy').shape

(268, 3)

In [ ]:

''' Make Label '''


In [29]:
#脳情報データの読み込み
#respTrn は training data ,respVal は test data
import h5py
f = h5py.File( '/home/dl-box/デスクトップ/PythonFile/nishimoto/SampleFullsn/fullsnspmResp.mat' , 'r' )
print( list( f.keys() ) )

['respTrn', 'respVal']


In [30]:
print( '・脳情報データのサイズ(train):' + str(f['respTrn'].shape) + ' ・脳情報データのサイズ(test):' + str(f['respVal'].shape) )

・脳情報データのサイズ(train):(3600, 32, 100, 100) ・脳情報データのサイズ(test):(270, 32, 100, 100)


In [39]:
#( f['respTrn'][0].flatten() ).reshape(32,100,100) == f['respTrn'][0]
#>>>True

In [23]:
tr_label = []
for i in range( 2, len(f['respTrn']) ):
    tr_label.append( f['respTrn'][i].flatten() )
    if i % 1000 == 0:
        print(i)
        
te_label = []
for i in range( 2, len(f['respVal']) ):
    te_label.append( f['respVal'][i].flatten() )
    
'''
tr_label = []
for i in range( len(f['respTrn']) ):
    tr_label.append( np.array( f['respTrn'][i].flatten(), dtype=np.float16 ) )
    if i % 1000 == 0:
        print(i)
        
te_label = []
for i in range( len(f['respVal']) ):
    te_label.append( np.array( f['respVal'][i].flatten(), dtype=np.float16 ) )

'''

1000
2000
3000


"\ntr_label = []\nfor i in range( len(f['respTrn']) ):\n    tr_label.append( np.array( f['respTrn'][i].flatten(), dtype=np.float16 ) )\n    if i % 1000 == 0:\n        print(i)\n        \nte_label = []\nfor i in range( len(f['respVal']) ):\n    te_label.append( np.array( f['respVal'][i].flatten(), dtype=np.float16 ) )\n\n"

In [24]:
np.save( '../Training_Data/training_label(345).npy', tr_label )
np.save( '../Training_Data/test_label(345).npy', te_label )
np.load( '../Training_Data/test_label(345).npy').shape

(268, 320000)

In [25]:
f.close()

In [3]:
import h5py
t_train = h5py.File( '/home/dl-box/デスクトップ/PythonFile/hirose/Training_Data/t_train.h5py' , 'w' )
t_test = h5py.File( '/home/dl-box/デスクトップ/PythonFile/hirose/Training_Data/t_test.h5py' , 'w' )

In [ ]:
for i in range(32):
    tr = []
    
    for j in range( 2, len(f['respTrn']) ):
        tr.append( np.r_[ f['respTrn'][j][i].flatten(), f['respTrn'][j-1][i].flatten(), f['respTrn'][j-2][i].flatten() ] )
        
    t_train[ 'Trn'+str(i+1) ] = np.array( tr )
    print( str(i) + 'スライスまで終了' )

    
    
    
for i in range(32):
    te = []
    
    for j in range( 2, len(f['respVal']) ):
        te.append( np.r_[ f['respVal'][j][i].flatten(), f['respVal'][j-1][i].flatten(), f['respVal'][j-2][i].flatten() ] )
        
    t_test[ 'Val'+str(i+1) ] = np.array( te )
    print( str(i) + 'スライスまで終了' )


In [12]:
t_train.close()
t_test.close()

In [10]:
import numpy as np
np.r_[ np.array([np.array([1,2,3]), np.array([1,2,3]), np.array([1,2,3,4])]) ]

array([array([1, 2, 3]), array([1, 2, 3]), array([1, 2, 3, 4])],
      dtype=object)

In [20]:
np.r_[ np.array([1,2,3]), np.array([1,2,3]), np.array([1,2,3]) ]

array([1, 2, 3, 1, 2, 3, 1, 2, 3])

In [30]:
tr_label = []
for i in range( 2, len(f['respTrn']) ):
    tr_label.append( np.r_[ f['respTrn'][i].flatten(), f['respTrn'][i-1].flatten(), f['respTrn'][i-2].flatten() ] )
    if i % 1000 == 0:
        print(i)
        
te_label = []
for i in range( 2, len(f['respVal']) ):
    te_label.append( np.r_[ f['respVal'][i].flatten(), f['respVal'][i-1].flatten(), f['respVal'][i-2].flatten() ] )

1000
2000
3000


In [35]:
np.save( '../Training_Data/t_train.npy', tr_label[:-1] )
np.save( '../Training_Data/t_test.npy', te_label[:-1] )

In [ ]:

'''文の構造'''


In [3]:
path = [ ('./text_index/'+os.path.basename( r )) for r in sorted( glob.glob( './text_index/*.npy' ) ) ]
s = []
m = []
for i in range(len(path)):
    a = np.load(path[i])
    m.append(len(a))
    l = 0
    for j in range(len(a)):
        if len(a[j]) >= l:
            l = len(a[j])
    s.append(l)

In [4]:
s = np.array(s)
m = np.array(m)

In [8]:
print( '平均：文の数 '+str(m.mean())+' 文の長さ '+str(s.mean()) )

平均：文の数 14.993669250645995 文の長さ 24.04108527131783


In [9]:
print( 'max：文の数 '+str(m.max())+' 文の長さ '+str(s.max()) )

max：文の数 28 文の長さ 50


In [10]:
print( 'min：文の数 '+str(m.min())+' 文の長さ '+str(s.min()) )

min：文の数 5 文の長さ 10


In [12]:
weights = np.load( './lstm/weights.npy' )

In [17]:
weights = np.array( weights, dtype=np.float16 )

In [18]:
np.save( './weights.npy', weights )

In [19]:
weights.shape

(129372, 300)

In [45]:
label = sorted(glob.glob('./padding_plus1/*.npy'))
x_train = []
x_test = []
for idx in range( 7, len(label) ):
    if idx%2 == 1:
        if idx < 7200:
            x_train.append( np.load(label[idx]) )
        elif idx >= 7207:
            x_test.append( np.load(label[idx]) )

In [46]:
np.save( '../Training_Data/x_train.npy', x_train )
np.save( '../Training_Data/x_test.npy', x_test )

In [47]:
np.load( '../Training_Data/x_test.npy' ).shape

(267, 18, 35)